# Produce IO Tables from OECD data

This notebook generalises the code from the notebook "Produce OECD IO tables - one country only" to multiple countries in one year. Its aim is to produce Input-Output (IO) tables for a number of countries using the OECD data published [here](https://www.oecd.org/sti/ind/inter-country-input-output-tables.htm). 

## 1. Wide format

In [5]:
# Imports and path
!pip install pymrio
import pymrio
import pandas as pd
from pathlib import Path
oecd_storage = Path('/project_data/data_asset/')

In [6]:
# Select the year to parse - 2015 in this case
oecd_path_year = pymrio.parse_oecd(path=oecd_storage, year=2015)

In [7]:
# Show countries
print('There are', len(oecd_path_year.get_regions()), 'countries in the dataset:')
print(set(oecd_path_year.get_regions()))

There are 65 countries in the dataset:
{'SGP', 'TUN', 'IDN', 'HKG', 'MLT', 'TUR', 'NOR', 'GRC', 'BGR', 'CHN', 'KHM', 'SVN', 'MYS', 'BEL', 'CZE', 'CHL', 'EST', 'KOR', 'NLD', 'DNK', 'LUX', 'KAZ', 'HRV', 'POL', 'PRT', 'BRN', 'PHL', 'THA', 'VNM', 'CRI', 'ARG', 'MEX', 'SWE', 'ROW', 'ZAF', 'SAU', 'HUN', 'NZL', 'FIN', 'IRL', 'DEU', 'CHE', 'ISL', 'MAR', 'GBR', 'CAN', 'FRA', 'ROU', 'BRA', 'COL', 'ISR', 'AUS', 'ESP', 'ITA', 'PER', 'RUS', 'USA', 'CYP', 'LVA', 'AUT', 'JPN', 'TWN', 'SVK', 'LTU', 'IND'}


In [8]:
countries = oecd_path_year.get_regions()
non_essential_sectors = ['07T08','24','25','29','30','68']

In [9]:
# Constrcut Input-Output tables for selected countries    
IO_EXT = oecd_path_year.Z.copy()
        
# TAXES AND VALUE ADDED
TAXSUB = oecd_path_year.factor_inputs.F.iloc[oecd_path_year.factor_inputs.F.index.get_level_values('inputtype') != 'VALU']
VA = oecd_path_year.factor_inputs.F.iloc[oecd_path_year.factor_inputs.F.index.get_level_values('inputtype') == 'VALU']
VA_TAXES = VA.append(TAXSUB * (-1))

IO_EXT.loc[('ALL', 'VALUE ADDED'),:] = pd.Series(VA_TAXES.sum()).values


In [10]:
# Collapse non-essential columns
country_NE = pd.DataFrame()

for country in countries:
    
    for sector in non_essential_sectors:
        
        column = IO_EXT[(country, sector)]
        country_NE = pd.concat([country_NE, column], axis = 1)
        IO_EXT.drop(columns = (country, sector), inplace = True)
    
    country_NE = country_NE.sum(axis = 1)
    
    IO_EXT[(country, 'All non-essential')] = country_NE
    
    country_NE = pd.DataFrame()

In [11]:
# Collapse non-essential rows
country_NE = pd.DataFrame()

for country in countries:
    
    for sector in non_essential_sectors:
        
        row = IO_EXT.loc[(country, sector),:]
        country_NE = pd.concat([country_NE, row], axis = 1)
        IO_EXT.drop(index = (country, sector), inplace = True)
    
    country_NE = country_NE.sum(axis = 1)
    
    IO_EXT.loc[(country, 'All non-essential'),:] = country_NE
    
    country_NE = pd.DataFrame()

In [12]:
# TOTAL PRODUCTION
IO_EXT.loc[('ALL', 'TOTAL PRODUCTION'),:] = pd.Series(IO_EXT.sum()).values   

In [13]:
IO_EXT

region                             ARG                             \
sector                           01T03         05T06           09   
region sector                                                       
ARG    01T03              13014.595854      8.049052     4.143001   
       05T06                199.321049   2276.678829   275.113858   
       09                   313.959066   1894.970278   246.435218   
       10T12               1960.104890     31.815860     5.132780   
       13T15                257.570464     39.774788     6.260411   
...                                ...           ...          ...   
TWN    All non-essential      0.687945      1.416402     0.288667   
USA    All non-essential      8.382023      8.336595     1.382335   
VNM    All non-essential      0.131674      0.124189     0.040799   
ZAF    All non-essential      0.614518      0.725694     0.140749   
ALL    TOTAL PRODUCTION   81625.463330  27843.740051  3748.790151   

region                                                             \
sector                           10T12         13T15           16   
region sector                                                       
ARG    01T03              34292.771836    964.672923   545.773454   
       05T06                254.403159     34.275764     7.246904   
       09                    39.008104      0.000000     2.405210   
       10T12               4438.490860    243.163011    22.569292   
       13T15                435.065366   6610.432902    26.232974   
...                                ...           ...          ...   
TWN    All non-essential      1.680883      0.348774     0.223092   
USA    All non-essential     13.529736      4.146243     1.715818   
VNM    All non-essential      0.130207      0.032578     0.015670   
ZAF    All non-essential      0.981533      0.247062     0.139073   
ALL    TOTAL PRODUCTION   87493.392897  27815.604696  7494.304599   

region                                                              \
sector                           17T18            19         20T21   
region sector                                                        
ARG    01T03                240.085264     16.713493    324.245670   
       05T06                 81.646943  16381.766599    757.582323   
       09                    12.716208     14.148393      6.509650   
       10T12                 88.041559     21.605191    173.685120   
       13T15                 76.357137     13.831488    136.254071   
...                                ...           ...           ...   
TWN    All non-essential      0.182624      0.147612      0.304447   
USA    All non-essential      2.447097      1.464879      7.432188   
VNM    All non-essential      0.042467      0.020378      0.302031   
ZAF    All non-essential      0.224580      1.151624      0.459515   
ALL    TOTAL PRODUCTION   14212.358454  30228.021257  23419.521919   

region                                  ...               SVK  \
sector                              22  ... All non-essential   
region sector                           ...                     
ARG    01T03                215.505180  ...          0.001767   
       05T06                101.862709  ...          0.000014   
       09                    22.030458  ...          0.000000   
       10T12                 50.899412  ...          0.004491   
       13T15                166.691463  ...          0.007941   
...                                ...  ...               ...   
TWN    All non-essential      0.683925  ...         20.721855   
USA    All non-essential      5.586056  ...         92.617364   
VNM    All non-essential      0.054106  ...          2.665338   
ZAF    All non-essential      0.440182  ...          5.229450   
ALL    TOTAL PRODUCTION   17230.765111  ...      42425.210363   

region                                 SVN               SWE  \
sector                   All non-essential All non-essential   
region sector                                 

In [14]:
# EXPORT TO CSV
#IO_EXT.to_csv('/project_data/data_asset/IO_WORLD.csv')

## 2. Long format

In [15]:
# Copy dataframe and prepare it for melting
IO_MELT = IO_EXT.copy()
IO_MELT.reset_index(inplace = True)
IO_MELT.rename(columns = {'region': 'input_country', 'sector': 'input_sector'}, inplace = True)

In [16]:
# Melt dataframe - From wide to long format
IO_MELT = pd.melt(IO_MELT, id_vars = ['input_country', 'input_sector'], var_name = ['output_country', 'output_sector'])
IO_MELT

,input_country,input_sector,output_country,output_sector,value
0,ARG,01T03,ARG,01T03,13014.595854
1,ARG,05T06,ARG,01T03,199.321049
2,ARG,09,ARG,01T03,313.959066
3,ARG,10T12,ARG,01T03,1960.104890
4,ARG,13T15,ARG,01T03,257.570464
...,...,...,...,...,...
4064250,TWN,All non-essential,ZAF,All non-essential,49.705607
4064251,USA,All non-essential,ZAF,All non-essential,212.045494
4064252,VNM,All non-essential,ZAF,All non-essential,4.406049
4064253,ZAF,All non-essential,ZAF,All non-essential,20863.262335


In [17]:
# EXPORT TO CSV
#IO_MELT.to_csv('/project_data/data_asset/IO_WORLD_LONG.csv', index = False)